In [1]:
import numpy as np
import pandas as pd
import requests
from time import sleep
import json
import math

In [2]:
API_KEY1 = "91ed49cd-f2bb-4bba-9ff8-0f4ac4584790" # основной ключ, надо поменять
API_KEY2 = "эээ ну это тут нужен ключ" # запасной ключ

#Обращения к API

In [3]:
BOX = {
    "Москва": (37.30, 37.95, 55.55, 55.95),
    "Санкт-Петербург": (30.10, 30.50, 59.80, 60.10)
}

CITIES = [
    "Москва", "Санкт-Петербург"
]

In [4]:
cities_df = pd.DataFrame(columns=["city", "lon_min", "lon_max", "lat_min", "lat_max"])
i=0

for city in CITIES:
  lon_min, lon_max, lat_min, lat_max = BOX[city]
  cities_df.loc[i] = [city, lon_min, lon_max, lat_min, lat_max]
  i+=1

cities_df.to_csv("cities.csv", index=False)
print("Сохранено как 'cities.csv'")


Сохранено как 'cities.csv'


In [5]:
cities_df = pd.read_csv("cities.csv")
cities_df

,city,lon_min,lon_max,lat_min,lat_max
0,Москва,37.3,37.95,55.55,55.95
1,Санкт-Петербург,30.1,30.50,59.80,60.10


In [6]:
import logging
logger1 = logging.getLogger("2GIS_API.region") # вернет один и тот же объект для этого имени в любом месте программы
logger1.setLevel(logging.INFO)  # отсекает все записи ниже INFO на уровне логгера. То есть debug не пройдет дальше по цепочке, а info, warning, error, critical будут обработаны

fh1 = logging.FileHandler("RegionLogs.log") # открывает файл в текущей рабочей директории и будет записывать туда строки

formatter1 = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s') # определяет, как будет выглядеть запись в логах
fh1.setFormatter(formatter1) # Назначает форматтер обработчику

logger1.addHandler(fh1) #добавляет обработчик логгеру

In [ ]:
def get_region_id(city: str) -> str:
  try:
      r = requests.get(
          "https://catalog.api.2gis.com/2.0/region/search",
          params={"q": city, "key": API_KEY1, "locale": "ru_RU"},
          headers={"accept": "application/json"},
          timeout=20,
      )
      r.raise_for_status() # проверка на ошибки типа 4xx и 5xx, если ккод не 200, то вызовет requests.exceptions.HTTPError
      # r.json() в случае ошибки будет таким: {'meta': {'api_version': '2.0.19411', 'code': 403, 'error': {'message': 'Authorization error, incorrect key.', 'type': 'forbidden'}, 'issue_date': '20251111'}}
      # r.json() в случае успеха будет таким: {'meta': {'api_version': '2.0.19411', 'code': 200, 'issue_date': '20251111'}, 'result': {'items': [{'id': '32', 'name': 'Москва', 'type': 'region'}], 'total': 1}}
      code = r.json()["meta"]["code"] # получаем 'code' из r.json()

      if code != 200: # если код не 200, то вызываем RuntimeError чтобы обработать ошибку
          message = r.json()["meta"]["error"]["message"]
          raise RuntimeError("RuntimeError")

      items = r.json()["result"]["items"]  # если код 200, то ищем id бара и передаем дальше
      logger1.info(f"Determined region_id for city {city} : {items[0]['id']} ")

      return items[0]["id"]

  except requests.exceptions.HTTPError as e:
      logger1.error(f"HTTPError occurred: '{e}' while determining region_id for city '{city}'")
  except RuntimeError as e:
      logger1.error(f"Error occurred: '{e}' while determining region_id for city '{city}'. code: {code}, message: {message}")
  except Exception as e:
      logger1.error(f"Unexpected error: '{e}' while determining region_id for city '{city}'")

def get_bar_rubric_id(region_id: str) -> str:
    try:
        r = requests.get(
            "https://catalog.api.2gis.com/2.0/catalog/rubric/search",
            params={"q": "бар", "region_id": region_id, "key": API_KEY1, "locale": "ru_RU"},
            headers={"accept": "application/json"},
            timeout=20,
        )
        r.raise_for_status()
        code = r.json()["meta"]["code"]
        # r.json() в случае ошибки будет таким: {'meta': {'api_version': '2.0.19411', 'code': 403, 'error': {'message': 'Authorization error, incorrect key.', 'type': 'forbidden'}, 'issue_date': '20251111'}}
        # r.json() в случае успеха будет таким: {'meta': {'api_version': '2.0.19411', 'code': 200, 'issue_date': '20251111'}, 'result': {'items': [{'alias': 'bary', 'branch_count': 2565, 'caption': 'Бары', 'id': '159', 'keyword': 'Бары', \
        # 'name': 'Бары', 'org_count': 2011, 'parent_id': '2', 'seo_name': 'Бары / Пабы', 'title': 'Бары', 'type': 'rubric'}, {'alias': 'barbershopy', 'branch_count': 2509, 'caption': 'Барбершопы', 'id': '110998',... и так далее все совпадения по слову "бар"

        if code != 200: # если код не 200, то вызываем RuntimeError чтобы обработать ошибку
            message = r.json()["meta"]["error"]["message"]
            raise RuntimeError("RuntimeError")

        items = r.json()["result"]["items"]
        for rb in items:
            # Строгое совпадение по alias исключает 'суши-бары', 'барбершопы' и пр.
            if (rb.get("alias") or "").lower() == "bary":
                logger1.info(f"Determined bar_id: {rb['id']}")
                return rb["id"]

        # fallback: на случай, если alias отсутствует — проверим точное 'name'
        for rb in items:
            if (rb.get("name") or "").strip().lower() in ("бар", "бары"):
                logger1.info(f"Determined bar_id: {rb['id']}")
                return rb["id"]

    except requests.exceptions.HTTPError as e:
        logger1.error(f"HTTPError occurred: '{e}' while determining bar_id for region_id: '{region_id}'")
    except RuntimeError as e:
        logger1.error(f"Error occurred: '{e}' while determining bar_id for region_id: '{region_id}'. code: {code}, message: {message}")
    except Exception as e:
        logger1.error(f"Unexpected error: '{e}' while determining bar_id for region_id: '{region_id}'")



# для нахождения bar_id, который нам понадобится далее, нам нужно определить region_id (так устроена апишка 2ГИСа). region_id ищем через обращение к API
# так как для всех регионов id рубрики "бар" одинаковое (выяснили на этапе подготовки), мы не станем циклом перебирать для каждого
# чтобы не расходовать ресурсы, время и запросы к API.

logger1.info("Started")

region_id = get_region_id(CITIES[0]) # ищем id региона для Москвы, можно выбрать любой город из нашего списка
bar_id = get_bar_rubric_id(region_id) # нашли bar_id и дальше переходим к коду, который будет успользовать этот айдишник для поиска баров по городам

logger1.info("Finished")

In [8]:
# тут все аналогично первому логгеру, но уровень ставим debug тк будет очень много несущественных оповещений, а так же задаем другой логгер - 2GIS_API.api
logger2 = logging.getLogger("2GIS_API.api")
logger2.setLevel(logging.DEBUG)

fh2 = logging.FileHandler("APILogs.log")

formatter2 = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh2.setFormatter(formatter2)

logger2.addHandler(fh2)

In [ ]:
# --- настройки под демо-лимиты ---
PAGE_SIZE = 10      # демо: 1-10
MAX_PAGES = 5       # демо: 5 страниц на точку
RADIUS_M = 2000     # радиус поиска в метрах
METERS_PER_DEG_LAT = 111320.0  # ~м/градус широты
#bar_id = 159 # предпологается, что был запущен предыдущий код и оттуда подтягивается bar_id, но если это не так, то можно сделать так

def is_bar_row(rubrics):
    # функция для проверки является ли нужным нам баром или пабом(так уж решили) строчка
    if not isinstance(rubrics, list):
        return False
    for r in rubrics:
        if not isinstance(r, dict):
            continue
        name = r.get("name", "").lower()
        if "бар" in name or "паб" in name:
            return True
    return False

def rotate_key(req_count: int, current_key: str) -> str:
    if req_count >= 999 and current_key == API_KEY1: # если по какой-то причине слишком много запросов, например много сетевых ошибок, то меняем ключ на запасной
        logger2.warning("Changed the keys") # warning потому что превышение запросов по одному ключу не входило в наши планы
        return API_KEY2
    return current_key

def fetch_circle(lon: float, lat: float, bar_id: str, req_count: int, api_key: str):
    out = []
    for page in range(1, MAX_PAGES + 1): # перебираем страницы для обращения к API
        api_key = rotate_key(req_count, api_key) # проверяем и меняем ключ если необходимо

        for attempt in range(3): # пробуем три раза на случай различных несущественных ошибок
            try:
                r = requests.get(
                    "https://catalog.api.2gis.com/3.0/items",
                    params={
                        "q": "бар",
                        "type": "branch",
                        "rubric_id": bar_id,
                        "point": f"{lon},{lat}",
                        "radius": RADIUS_M,
                        "page": page,
                        "page_size": PAGE_SIZE,
                        "key": api_key,
                        "locale": "ru_RU",
                        "fields": "items.point,items.address,items.rubrics,items.reviews,items.attribute_groups", # добавил "items.attribute_groups"
                    },
                    headers={"accept": "application/json"},
                    timeout=20,
                )
                req_count += 1 # увеличиваем текущий счетчик запросов

                r.raise_for_status() # далее как в предыдущем коде обработка ошибок
                code = r.json()["meta"]["code"]

                if code == 200:
                    logger2.debug("Code 200")
                    break
                else:
                    message = r.json()["meta"]["error"]["message"]
                    if code == 404: # в выбранном секторе не обязательно есть бары, тогда апишка вернет 404
                        logger2.warning(f"API request returned error code {code}, message: '{message}' - probably no bars in this area")
                    elif code != 200: # если проблема, например с загрузкой, то пробуем еще раз
                        wait_s = attempt + 1
                        logger2.error(f"API request error: code {code}, message: '{message}'. Waiting {wait_s} s and retrying, attempt {attempt+1}/3")
                        sleep(wait_s)
                        continue
                    else:
                        logger2.error(f"Unexpected API response error: code {code}, message: '{message}'")
                    break

            except requests.exceptions.HTTPError as e:
                logger2.error(f"HTTP error during API request: {e}")
            except Exception as e:
                logger2.error(f"Unexpected error during API request: {e}")
        else:
            # все попытки провалились - выходим из пагинации этого круга
            break

        if code == 200: # если код 200, то получаем и записываем нужные нам результаты
            items = r.json()["result"]["items"]
            if not items:
                break

            out.extend(items)
            if len(items) < PAGE_SIZE: # если предметов меньше чем мы запросили, то, вероятно, следующая страница будет пустой, значит нет смысла к ней обращаться
                break
        elif code == 404: # если код 404, те нет баров в этой области, то нет смысла листать страницы
            break
        sleep(0.15)

    return out, req_count, api_key


# наша идея: разбить города на области с радиусом RADIUS_M и собирать по этим областям
# для этого нужно для каждого города разметить центры.
# логика выбора центров: выбираем с небольшим наслаиванием нижнего на верхние, чтобы достичь максмимального покрытия

# Вертикальный шаг по широте:
# для начала считаем distlat_deg - это шаг в градусах по широте эквивалентный 3км - нашему радиусу.
# distlat_deg = RADIUS_M / METERS_PER_DEG_LAT
# Это примерно 1 радиус в градусах широты, потому что 1 градус широты почти всегда равен 111.32 км
# Значит соседние ряды центров по вертикали идут через 1 радиус. Это дает сильное перекрытие по вертикали

# Горизонтальный шаг по долготе для каждого ряда на широте lat:
# distlon_deg = 2 * RADIUS_M / (METERS_PER_DEG_LAT * math.cos(math.radians(lat)))
# distlon_deg считается так, потому что чем севернее, тем меньше градусов нужно на те же метры (придумали не сами - заимствовано из интернета)
# шаг по горизонтали равен 2R потому что огризонтальные окружности не должны наслаиваться

# Чередование рядов:
# Для четных рядов старт по долготе идет от lon_min.
# Для нечетных рядов старт сдвигается на полшага: start_lon = lon_min + 0.5 * distlon_deg.
# Это сдвигает ряды на один радиус вправо и формирует почти шестиугольную упаковку, чтобы закрыть зазоры между кругами соседних рядов.

# Генерация точек:
# Внутренние циклы добавляют центры (lat, lon) пока lat <= lat_max и lon <= lon_max с маленькой эпсилон-поблажкой + 1e-9, чтобы не терять граничные точки из-за погрешностей.

# Важно! сначала считаем lat, а потом lon потому что так удобнее считать)
logger2.info("Started")

distlat_deg = RADIUS_M/METERS_PER_DEG_LAT # шаг в градусах по широте эквивалентный 3км, те примерно 0.02695
req_count = 0
api_key = API_KEY1

frames = [] # сюда будем добавлять то что получили из API

for i in range(cities_df.shape[0]):
    city, lon_min, lon_max, lat_min, lat_max = cities_df.iloc[i]

    logger2.info(f"Processing city: {city}. API requests so far: {req_count}.")

    centers = [] # тут определяем центры как было расписано ранее
    row = 0
    lat = lat_min
    while lat <= lat_max + 1e-9:
        distlon_deg = 2 * RADIUS_M / (METERS_PER_DEG_LAT * math.cos(math.radians(lat)))
        start_lon = lon_min + (0.5 * distlon_deg if (row % 2) else 0.0)
        lon = start_lon
        while lon <= lon_max + 1e-9:
            centers.append((lat, lon))
            lon += distlon_deg
        row += 1
        lat += distlat_deg

    logger2.debug(f"Number of centers for city '{city}': {len(centers)}")

    collected = {} # сюда складываем временные данные, которые получим после прохода по всем центрам города
    j=0
    for (lat, lon) in centers:
        logger2.debug(f"Processing center #{j}, {len(centers) - j} centers remaining")
        j+=1
        # Важно: порядок аргументов point это lon,lat
        objs, req_count, api_key = fetch_circle(lon, lat, bar_id, req_count, api_key)

        logger2.debug(f"Current request count: {req_count}")
        for it in objs:
            if isinstance(it, dict) and "id" in it:
                collected[it["id"]] = it

    df_city = pd.json_normalize(list(collected.values()))
    if not df_city.empty:
        if "rubrics" in df_city: # теперь из всего что сохраняли в collected ищем только бары и пабы по bar_id через функцию is_bar_row()
            df_city = df_city[df_city["rubrics"].apply(is_bar_row)]
        df_city["city"] = city # записываем город
        frames.append(df_city) # сохраняем это в список frames

    #logger2.info(f"Collected {len(frames[i])} rows")

df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
df.to_csv("bars_top_cities.csv", index=False)
print("CSV сохранен: bars_top_cities.csv")
logger2.info("Finished")

# {'address': {'building_id': '2674647955915599', 'building_name': 'Порт Артур', 'components': \
# [{'number': '22а', 'street': 'Южный бульвар', 'street_id': '2674751013132960', 'type': 'street_number'}], 'postcode': '603123'}, 'address_comment': \
#'3 этаж; через лифт', 'address_name': 'Южный бульвар, 22а', 'id': '70000001027850910', 'name': 'Butch&dutch, панорамный ресторан', 'point': {'lat': 56.21686, 'lon': 43.866138}, 'reviews': \
#{'general_rating': 3.3, 'general_review_count': 197, 'general_review_count_with_stars': 250, 'is_reviewable': True, 'is_reviewable_on_flamp': True, 'items': [{'is_reviewable': True, 'tag': '2gis_reviews'},\
# {'is_reviewable': True, 'rating': 3.6, 'review_count': 5, 'tag': 'flamp'}], 'org_rating': 4.5, 'org_review_count': 5768, 'org_review_count_with_stars': 23989, 'rating': 3.6, 'review_count': 5}, 'rubrics': \
#[{'alias': 'bary', 'id': '159', 'kind': 'additional', 'name': 'Бары', 'parent_id': '2', 'short_id': 159}, {'alias': 'restorany', 'id': '164', 'kind': 'primary', 'name': 'Рестораны', 'parent_id': '2', 'short_id': 164},\
# {'alias': 'karaoke_zaly', 'id': '21387', 'kind': 'additional', 'name': 'Караоке-залы', 'parent_id': '2', 'short_id': 21387}, {'alias': 'dostavka_edy', 'id': '1203', 'kind': 'additional', 'name': 'Доставка еды', \
#'parent_id': '2', 'short_id': 1203}, {'alias': 'banketnye_zaly', 'id': '10803', 'kind': 'additional', 'name': 'Банкетные залы', 'parent_id': '2', 'short_id': 10803}], 'type': 'branch'}
#
# те для этой штуки запишется так:
# {'70000001027850910': {'address': {'building_id': '2674647955915599',...

CSV сохранен: bars_top_cities.csv
